In [0]:
# Import libraries
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType

In [0]:
# Define the paths for Pinterest post, geolocation, and user data in your S3 bucket
pin_data_path = "/mnt/0e6999790cc9_mount_name/topics/0e6999790cc9.pin/partition=0/*.json"
geo_data_path = "/mnt/0e6999790cc9_mount_name/topics/0e6999790cc9.geo/partition=0/*.json"
user_data_path = "/mnt/0e6999790cc9_mount_name/topics/0e6999790cc9.user/partition=0/*.json"

# Read JSON files into DataFrames
df_pin = spark.read.json(pin_data_path)
df_geo = spark.read.json(geo_data_path)
df_user = spark.read.json(user_data_path)

# Replace empty entries and entries with no relevant data with Nones
df_pin_cleaned = df_pin.na.fill("None")

# Perform transformations on the follower_count column
df_pin_cleaned = df_pin_cleaned.withColumn("follower_count", df_pin_cleaned["follower_count"].cast(IntegerType()))

# Ensure that each column containing numeric data has a numeric data type
numeric_columns = ["follower_count"]  
for col in numeric_columns:
    df_pin_cleaned = df_pin_cleaned.withColumn(col, df_pin_cleaned[col].cast(IntegerType()))

# Clean the data in the save_location column to include only the save location path
df_pin_cleaned = df_pin_cleaned.withColumn("save_location", F.expr("substring_index(save_location, '/', -1)"))

# Rename the index column to ind
df_pin_cleaned = df_pin_cleaned.withColumnRenamed("index", "ind")

# Reorder the DataFrame columns
desired_column_order = ["ind", "unique_id", "title", "description", "follower_count", "poster_name",
                        "tag_list", "is_image_or_video", "image_src", "save_location", "category"]
df_pin_cleaned = df_pin_cleaned.select(desired_column_order)

# Show the cleaned and transformed DataFrame
df_pin_cleaned.show()

from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, FloatType, TimestampType

# Create a new column 'coordinates' based on 'latitude' and 'longitude'
df_geo = df_geo.withColumn('coordinates', F.array('latitude', 'longitude'))

# Drop 'latitude' and 'longitude' columns
df_geo = df_geo.drop('latitude', 'longitude')

# Convert 'timestamp' column from string to timestamp data type
df_geo = df_geo.withColumn('timestamp', F.to_timestamp('timestamp'))

# Reorder DataFrame columns
desired_column_order_geo = ["ind", "country", "coordinates", "timestamp"]
df_geo = df_geo.select(desired_column_order_geo)

# Show the cleaned and transformed DataFrame
df_geo.show()

from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType

# Create a new column user_name by concatenating first_name and last_name
df_user = df_user.withColumn("user_name", F.concat_ws(" ", "first_name", "last_name"))

# Drop the first_name and last_name columns
df_user = df_user.drop("first_name", "last_name")

# Convert the date_joined column from string to timestamp
df_user = df_user.withColumn("date_joined", F.to_timestamp("date_joined", "yyyy-MM-dd HH:mm:ss"))

# Reorder the DataFrame columns
desired_column_order_user = ["ind", "user_name", "age", "date_joined"]
df_user = df_user.select(desired_column_order_user)

# Show the cleaned and transformed DataFrame
df_user.show()

# 1. Find the most popular category people post to based on their country.
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Join df_pin_cleaned and df_geo on the 'ind' column
df_combined = df_pin_cleaned.join(df_geo, 'ind', 'inner')

# Group by country and category, count the occurrences, and create a new column 'category_count'
df_category_count = df_combined.groupBy('country', 'category').agg(F.count('ind').alias('category_count'))

# Use Window function to rank categories within each country based on count
window_spec = Window.partitionBy('country').orderBy(F.desc('category_count'))
df_category_ranked = df_category_count.withColumn('rank', F.rank().over(window_spec))

# Filter for the top-ranked category within each country
df_most_popular_category = df_category_ranked.filter(F.col('rank') == 1).drop('rank')

# Show the result
df_most_popular_category.show()

# 2. Find which was the most popular category each year
from pyspark.sql.functions import col, year, count
from pyspark.sql.window import Window

# Join df_pin_cleaned and df_geo on the 'ind' column
df_combined = df_pin_cleaned.join(df_geo, 'ind', 'inner')

# Create a new column 'post_year' based on the 'timestamp' column
df_combined = df_combined.withColumn("post_year", year("timestamp"))

# Filter posts between 2018 and 2022
df_filtered_posts = df_combined.filter((col("post_year") >= 2018) & (col("post_year") <= 2022))

# Group by post_year, category, and count the occurrences
df_category_count = df_filtered_posts.groupBy("post_year", "category").agg(count("*").alias("category_count"))

# Show the result
df_category_count.show()

# 3. Find the user with the most followers in each country

from pyspark.sql.functions import max, rank, desc

# Find the user with the most followers in each country
df_most_followers_by_country = df_pin.join(df_geo, df_pin["index"] == df_geo["ind"]) \
    .groupBy('country', 'poster_name') \
    .agg(max('follower_count')) \
    .withColumn('rank', rank().over(Window.partitionBy('country').orderBy(desc('max(follower_count)')))) \
    .filter('rank = 1') \
    .select('country', 'poster_name', 'max(follower_count)') \
    .withColumnRenamed('max(follower_count)', 'follower_count')

df_most_followers_by_country.show()

# Find the country with the user that has the most followers
df_most_followers_country = df_most_followers_by_country.groupBy('country') \
    .agg(max('follower_count').alias('follower_count')) \
    .orderBy(desc('follower_count')) \
    .limit(1) \
    .select('country', 'follower_count')

df_most_followers_country.show()

# 4. Find the most popular category for different age groups

from pyspark.sql.functions import col, when

# Define age groups
age_groups = {
    '18-24': (18, 24),
    '25-35': (25, 35),
    '36-50': (36, 50),
    '+50': (51, float('inf'))  # Assuming 50 and above as '+50'
}

# Create a new column 'age_group' based on the 'age' column
df_user_with_age_group = df_user.withColumn(
    'age_group',
    when((col('age') >= age_groups['18-24'][0]) & (col('age') <= age_groups['18-24'][1]), '18-24')
    .when((col('age') >= age_groups['25-35'][0]) & (col('age') <= age_groups['25-35'][1]), '25-35')
    .when((col('age') >= age_groups['36-50'][0]) & (col('age') <= age_groups['36-50'][1]), '36-50')
    .when((col('age') >= age_groups['+50'][0]), '+50')
    .otherwise('Unknown')
)

# Join df_pin_cleaned and df_geo on the 'ind' column
df_combined = df_pin_cleaned.join(df_geo, 'ind', 'inner')

# Join the combined DataFrame with df_user_with_age_group on the 'ind' column
df_combined_with_age_group = df_combined.join(df_user_with_age_group, 'ind', 'inner')

# Group by age_group, category, and count the occurrences
df_category_count_by_age_group = df_combined_with_age_group.groupBy('age_group', 'category').agg(count('*').alias('category_count'))

# Show the result
df_category_count_by_age_group.show()

# 5. Find the median follower count for different age groups

from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Define the age groups
age_groups = ["18-24", "25-35", "36-50", "+50"]

# Create a DataFrame with the age groups
df_age_groups = spark.createDataFrame([(group,) for group in age_groups], ["age_group"])

# Define conditions for age groups
conditions = [
    (F.col("age") >= 18) & (F.col("age") <= 24),
    (F.col("age") >= 25) & (F.col("age") <= 35),
    (F.col("age") >= 36) & (F.col("age") <= 50),
    (F.col("age") > 50)
]

# Use the conditions to create the age_group column
df_user = df_user.withColumn("age_group", F.when(conditions[0], "18-24")
                                       .when(conditions[1], "25-35")
                                       .when(conditions[2], "36-50")
                                       .when(conditions[3], "+50")
                                       .otherwise(None))

# Join df_user and df_pin_cleaned on a common column, e.g., ind
df_user_with_follower_count = df_user.join(df_pin_cleaned, "ind", "inner")

# Join df_user_with_follower_count and df_age_groups to include age_group information
df_user_with_follower_and_age = df_user_with_follower_count.join(df_age_groups, "age_group", "right_outer")

# Calculate the median follower count for each age group
df_median_follower_count = df_user_with_follower_and_age.groupBy("age_group") \
    .agg(F.expr("percentile(follower_count, 0.5)").alias("median_follower_count"))

# Show the result
df_median_follower_count.show()

# 6. Find how many users have joined each year

from pyspark.sql.functions import year, count

# Create a new column 'post_year' based on the 'date_joined' column
df_user_joined = df_user.withColumn("post_year", year("date_joined"))

# Filter users who joined between 2015 and 2020
df_filtered_users = df_user_joined.filter((col("post_year") >= 2015) & (col("post_year") <= 2020))

# Group by post_year and count the occurrences
df_users_joined_count = df_filtered_users.groupBy("post_year").agg(count("*").alias("number_users_joined"))

# Show the result
df_users_joined_count.show()

# 7. Find the median follower count of users based on their joining year

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Assuming df_user_with_follower_and_age is the DataFrame containing user information
# Make sure to replace it with the actual DataFrame you have

# Create the post_year column
df_filtered_users = df_user_with_follower_and_age.withColumn("post_year", F.year("date_joined"))

# Filter users who joined between 2015 and 2020
df_filtered_users = df_filtered_users.filter((F.col("post_year") >= 2015) & (F.col("post_year") <= 2020))

# Calculate the median follower count for each post_year
df_median_follower_count = df_filtered_users.groupBy("post_year") \
    .agg(F.expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count"))

df_median_follower_count.show()

# 8. Find the median follow count of users based on their joining year and age group

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Assuming df_user_with_follower_and_age is the DataFrame containing user information
# Make sure to replace it with the actual DataFrame you have

# Create the post_year column
df_filtered_users = df_user_with_follower_and_age.withColumn("post_year", F.year("date_joined"))

# Filter users who joined between 2015 and 2020
df_filtered_users = df_filtered_users.filter((F.col("post_year") >= 2015) & (F.col("post_year") <= 2020))

# Calculate the median follower count for each age group and post_year
df_median_follower_count_by_age_group = df_filtered_users.groupBy("age_group", "post_year") \
    .agg(F.expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count"))

df_median_follower_count_by_age_group.show()


# Display the schema of df_pin
print("Schema of df_pin:")
df_pin.printSchema()

# Display the schema of df_geo
print("Schema of df_geo:")
df_geo.printSchema()

df_user_joined.printSchema()

+--------------------+--------------------+---------------+
 country| poster_name| follower_count|
+--------------------+--------------------+---------------+
 Afghanistan| AllPosters| 72k|
 Albania| User Info Error|User Info Error|
 Algeria| YourTango| 942k|
 American Samoa| Mamas Uncut| 8M|
 Andorra| Glaminati| 799k|
 Angola| Tastemade| 8M|
 Anguilla|Kristen | Lifesty...| 92k|
Antarctica (the t...| HikenDip| 501k|
 Antigua and Barbuda| rhonda_floyd| 3k|
 Argentina| Next Luxury| 800k|
 Armenia|No Hurry To Get Home| 9k|
 Armenia| Pacho Tattoo| 9k|
 Aruba| Linda On The Run| 9k|
 Australia|Write Your Story ...| 5k|
 Austria|The World Pursuit...| 89k|
 Azerbaijan| Style Me Pretty| 6M|
 Bahamas| Andrew Martin| 5k|
 Bahrain|R.J. Weiss at The...| 46k|
 Bangladesh|Better Homes and ...| 4M|
 Barbados|The Creativity Ex...| 410k|
+--------------------+--------------------+---------------+
only showing top 20 rows

+-------+---------------+
country| follower_count|
+-------+---------------+
Albania|User Info Error|
+-------+---------------+